In [2]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

#카메라로 찍은 frame 을 띄워줄 image 객체 생성, 크기를 맞출 필요는 없다.
image = ipywidgets.Image(format='jpeg', width=300, height=300) 
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

print(camera.value)

display(image)

[[[163 157 149]
  [170 158 162]
  [166 155 155]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[158 158 149]
  [163 157 148]
  [161 155 154]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[168 154 157]
  [166 155 155]
  [165 156 151]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[ 42  48  50]
  [ 47  45  43]
  [ 47  41  43]
  ...
  [ 42  29  19]
  [ 39  37  35]
  [ 40  32  25]]

 [[ 49  47  45]
  [ 48  44  48]
  [ 53  45  47]
  ...
  [ 29  36  20]
  [ 33  35  27]
  [ 43  36  24]]

 [[ 53  44  43]
  [ 48  43  51]
  [ 46  48  48]
  ...
  [ 41  36  26]
  [ 35  36  24]
  [ 35  39  36]]]


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## 멈추는 코드

In [2]:
# 기존의 링크와 관찰자 제거 함수
def cleanup():
    global camera
    try:
        # 모든 관찰자 제거
        camera.unobserve_all()
        
        # 카메라 정지
        camera.stop()
    except Exception as e:
        print(f"Error during cleanup: {e}")
    finally:
        # 카메라 객체 삭제
        del camera
    
# 기존의 링크와 관찰자 제거, 로봇 제어파트 종료
cleanup()

## mqtt로 카메라 이미지 실행하는 코드
- 쓰레드 사용

In [4]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import paho.mqtt.client as mqtt
import threading
import time
import cv2

# MQTT 클라이언트 설정
broker_address = "192.168.110.103"
topic = "agv0/image"

client = mqtt.Client()
client.connect(broker_address)

# JetBot 카메라 설정
camera = Camera.instance(width=300, height=300)

# 카메라로 찍은 frame을 띄워줄 image 객체 생성, 크기를 맞출 필요는 없다.
image = ipywidgets.Image(format='jpeg', width=300, height=300)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

def publish_camera_frame():
    while True:
        frame = camera.value
        if frame is not None:
            img_str = cv2.imencode('.jpg', frame)[1].tobytes()
            client.publish(topic, img_str)
        time.sleep(0.1)  # 100ms 주기로 전송

# MQTT 전송을 위한 스레드 시작
thread = threading.Thread(target=publish_camera_frame)
thread.start()


timeout: timed out

## MQTT로 영상 보내기
- 객체로 보내기

In [4]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import paho.mqtt.client as mqtt
import time
import cv2
import numpy as np

# MQTT 클라이언트 설정
broker_address = "192.168.110.103"
topic = "agv0/image"

client = mqtt.Client()
client.connect(broker_address)

# JetBot 카메라 설정
camera = Camera.instance(width=300, height=300)

# 카메라로 찍은 frame을 띄워줄 image 객체 생성, 크기를 맞출 필요는 없다.
image = ipywidgets.Image(format='jpeg', width=300, height=300)
display(image)

# MQTT 전송 및 이미지 처리 함수
def execute(camera_image):
    frame = np.copy(camera_image)
    if frame is not None:
        img_str = cv2.imencode('.jpg', frame)[1].tobytes()
        client.publish(topic, img_str)
    return bgr8_to_jpeg(frame)

# 위젯에 연결하여 실시간으로 보여주기
time.sleep(1)
traitlets.dlink((camera, 'value'), (image, 'value'), transform=execute)

Image(value=b'', format='jpeg', height='300', width='300')

## MQTT 영상 받는 코드 

In [4]:
import paho.mqtt.client as mqtt
import ipywidgets as widgets
from IPython.display import display
import cv2
import numpy as np

# MQTT 브로커 설정
broker_address = "192.168.110.103"
topic = "agv0/image"

# 이미지 디스플레이 위젯 생성
image_widget = widgets.Image(format='jpeg', width=300, height=300)
display(image_widget)

# 수신한 메시지를 처리하는 콜백 함수
def on_message(client, userdata, message):
    # 메시지에서 이미지를 읽어들임
    img_data = np.frombuffer(message.payload, dtype=np.uint8)
    frame = cv2.imdecode(img_data, cv2.IMREAD_COLOR)
    
    # 이미지를 JPEG 형식으로 변환하여 위젯에 표시
    _, jpeg = cv2.imencode('.jpg', frame)
    image_widget.value = jpeg.tobytes()

# MQTT 클라이언트 생성 및 설정
client = mqtt.Client()
client.on_message = on_message

# 브로커 연결 및 구독
client.connect(broker_address)
client.subscribe(topic)

# MQTT 클라이언트 시작
client.loop_start()

Image(value=b'', format='jpeg', height='300', width='300')

## MQTT로 프레임 받고 라인 트레이싱

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import paho.mqtt.client as mqtt
import cv2
import numpy as np
import time

# IPython Libraries for display and widgets
from jetbot import Robot, bgr8_to_jpeg
from SCSCtrl import TTLServo

# JetBot 로봇 객체 생성
robot = Robot()

# 서보모터 준비
TTLServo.servoAngleCtrl(1, 10, 1, 150)
TTLServo.servoAngleCtrl(5, -40, 1, 200)  # 카메라 라인 보기 위해 고개 숙이기

# 이미지 표시를 위한 위젯 생성
image_widget = widgets.Image(format='jpeg', width=300, height=300)
target_widget = widgets.Image(format='jpeg', width=300, height=300)
x_moment_label = widgets.Label(value="x_moment: 0")
display(widgets.VBox([widgets.HBox([image_widget, target_widget]), x_moment_label]))

# MQTT 클라이언트 설정
broker_address = "192.168.110.103"
topic = "agv0/image"

client = mqtt.Client()

# Calibration 파라미터
intrinsic_parameter_path = "./parameters/intrinsic_parameter.npy"
dist_coeffs_path = "./parameters/distortion_coefficients.npy"
intrinsic_parameter = np.load(intrinsic_parameter_path)
dist_coeffs = np.load(dist_coeffs_path)

# 특정 RGB, HSV 범위 설정 (필요에 따라 조정)
lower_rgb = np.array([0, 0, 0])
upper_rgb = np.array([100, 100, 100])
lower_hsv = np.array([0, 0, 0])
upper_hsv = np.array([180, 66, 117])

# ROI에서 구할 무게중심, 제어에서 쓰일 변수
x_moment, y_moment = 150, 150
flag = 0
mid = 112

# 왜곡 보정해주는 함수
def callibration(img):
    global intrinsic_parameter, dist_coeffs
    undistorted_image = cv2.undistort(img, intrinsic_parameter, dist_coeffs)
    return undistorted_image

# ROI 추출 함수
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:  # Color 이미지인 경우
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    cv2.fillPoly(mask, vertices, ignore_mask_color)  # 다각형 내부를 흰색으로 채우기
    roi_image = cv2.bitwise_and(img, mask)           # 입력 이미지와 마스크 비트 연산
    return roi_image 

# 검은색만 마스킹하는 함수(RGB)
def mask_rgb(img, lower_rgb, upper_rgb):
    mask_black = cv2.inRange(img, lower_rgb, upper_rgb)
    masked_img = cv2.bitwise_and(img, img, mask=mask_black)
    return masked_img

# 카메라 프레임을 업데이트하는 함수
def execute(camera_image):
    global x_moment, y_moment, flag, mid

    # 프레임 가져오기
    frame = np.copy(camera_image)
    
    # 캘리브레이션 적용
    undistorted_frame = callibration(frame)

    # 원본 이미지의 해상도 확인
    height, width = undistorted_frame.shape[:2]

    # 새로운 사다리꼴 모양의 vertices 정의 (300x300 해상도 기준)
    vertices = np.array([[(width * 0.1, height), (width * 0.3, height * 0.3), (width * 0.7, height * 0.3), (width * 0.9, height)]], dtype=np.int32)

    # ROI 마스크 적용
    roi_frame = region_of_interest(undistorted_frame, vertices)
    mid = int(roi_frame.shape[0]/2)
    
    # RGB 마스킹 적용
    masked_frame = mask_rgb(roi_frame, lower_rgb, upper_rgb)

    # 그레이스케일 변환
    gray_frame = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)

    # Gaussian Blur
    blur_frame = cv2.GaussianBlur(gray_frame, (7, 7), 0)

    # Threshold 적용, 흑백으로 변환
    _, threshold_frame = cv2.threshold(blur_frame, 15, 255, cv2.THRESH_BINARY)

    # 외곽선 찾기
    outline_frame = cv2.findContours(threshold_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # 가장 바깥쪽 외곽선만 찾음
    outline_list = imutils.grab_contours(outline_frame)

    # 외곽선이 있을때
    frame_ds = np.zeros((300, 300, 3), dtype=np.uint8)
    if (len(outline_list)!=0):
        c = max(outline_list, key=cv2.contourArea)
        cv2.drawContours(frame_ds,[c],-1,(0,255,255),2)     # 외곽선을 노란색으로 그린다.

        # 무게중심 찾기 -> 제일 큰 도형의 무게중심
        M = cv2.moments(c)                                  # 외곽선 'c'의 모멘트 계산
        if (M['m00']!=0):
            x_moment = int(M['m10']/M['m00'])
            y_moment = int(M['m01']/M['m00'])
            frame_ds = cv2.circle(frame_ds, (x_moment, y_moment), 3, (255,0,0), -1)    # 무게중심 이미지에 그리기
    else:
        x_moment = 150

    # x_moment 값을 레이블에 반영
    x_moment_label.value = f"x_moment: {x_moment} , flag : {flag}, mid: {mid}, ROI shape: {roi_frame.shape} "

    # 프레임을 JPEG로 변환하여 위젯에 설정
    jpeg_image = bgr8_to_jpeg(frame_ds)
    
    ##########################################
    # 제어 파트 작성 부분 
    # left
    if  x_moment< (mid - 40):
        robot.set_motors(0.2, 1.0)
        flag=1

    elif x_moment < (mid - 20):
        robot.set_motors(0.2, 0.7)
        flag = 2

    # center    
    elif x_moment < (mid + 20):
        robot.set_motors(1.0, 1.0)
        flag = 3

    # right
    elif x_moment < (mid + 40):
        robot.set_motors(0.7, 0.2)
        flag = 4
        
    elif x_moment>= (mid + 40):
        robot.set_motors(1.0, 0.2)
        flag = 5

    return jpeg_image

def on_message(client, userdata, message):
    img_data = np.frombuffer(message.payload, dtype=np.uint8)
    frame = cv2.imdecode(img_data, cv2.IMREAD_COLOR)
    image_widget.value = bgr8_to_jpeg(frame)
    target_widget.value = execute(frame)

# MQTT 콜백 설정
client.on_message = on_message

# MQTT 브로커 연결 및 구독 설정
client.connect(broker_address)
client.subscribe(topic)
client.loop_start()

# 무한 루프를 사용하여 스크립트가 종료되지 않도록 함
while True:
    time.sleep(1)

## MQTT 수신 테스트

In [ ]:
import paho.mqtt.client as mqtt
import ipywidgets as widgets
from IPython.display import display

# 위젯 설정
button_command_label = widgets.Label(value="button command: unknown")
voice_command_label = widgets.Label(value="voice command: unknown")


topic_voice_command = "agv0/voiceCommand"  # voice
topic_button_command = "agv0/command" 

# MQTT 콜백 함수 설정
def on_connect(client, userdata, flags, reason_code, properties=None):
    
    global topic_voice_command, topic_button_command
    print("Connected with result code: {}".format(reason_code))
    client.subscribe(topic_voice_command)
    client.subscribe(topic_button_command)

def on_message(client, userdata, message):
    
    global topic_voice_command, topic_button_command
    
    if  (message.topic == topic_voice_command):
        voice_command_label.value = f"Topic: {message.topic} | Message: {message.payload.decode('utf-8')}"
    
    elif  (message.topic == topic_button_command):
        button_command_label.value = f"Topic: {message.topic} | Message: {message.payload.decode('utf-8')}"

    
    
# MQTT 클라이언트 설정
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message
broker_address = "192.168.110.103"
mqttc.connect(broker_address)

# 비동기 루프 시작
mqttc.loop_start()
display(button_command_label, voice_command_label)
